In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import time

In [3]:
from pysmFISH.pipeline import Pipeline
from pysmFISH import processing_cluster_setup

/home/simone/mini/envs/test_d/lib/python3.8/site-packages/dask_jobqueue/core.py:19: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/home/simone/mini/envs/test_d/lib/python3.8/site-packages/dask_jobqueue/core.py:19: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/home/simone/mini/envs/test_d/lib/python3.8/site-packages/dask_jobqueue/htcondor.py:6: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import parse_bytes


# AMEXP20200914_Spinal_cordGel1

## Correct the dataset

In [ ]:
# Correct the dataset file
from pysmFISH.data_models import Dataset


In [ ]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/AMEXP20200914_Spinal_cordGel1')
dataset_name = '211121_11_11_48_AMEXP20200914_Spinal_cordGel1_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'unmanaged_cluster'

In [ ]:
data = Dataset()
data.load_dataset(experiment_fpath / dataset_name)

In [ ]:
data.dataset.loc[:,'pipeline'] = 'smfish-serial-spinal-cord'
data.dataset.loc[:,'experiment_type'] = 'smfish-serial'
data.dataset.loc[:,'stitching_type'] = 'nuclei'
data.dataset.loc[:,'stitching_channel'] = 'DAPI'
data.dataset.loc[data.dataset.channel == 'DAPI','processing_type'] = 'nuclei'
data.dataset = data.dataset.loc[~(data.dataset.channel == 'Europium'),:]
data.dataset.loc[:,'machine'] = 'ROBOFISH1'

In [ ]:
data.save_dataset(data.dataset, experiment_fpath / 'Corrected_AMEXP20200914_Spinal_cordGel1_img_data_dataset.parquet' )

## Test run specific fovs

In [ ]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/AMEXP20200914_Spinal_cordGel1')
dataset_name = 'Corrected_AMEXP20200914_Spinal_cordGel1_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'unmanaged_cluster'
start_from_preprocessed_imgs =True

In [ ]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        scheduler_port= 22517,
        dataset_path = experiment_fpath / dataset_name,
        start_from_preprocessed_imgs=start_from_preprocessed_imgs)

In [ ]:
%%time
running_pipeline.run_setup()
running_pipeline.run_cluster_activation()
running_pipeline.run_parsing()
running_pipeline.run_required_steps()

In [ ]:
running_pipeline.client

In [ ]:
import numpy as np

In [ ]:
number_fovs_process = 1
selected_fovs = np.random.choice(running_pipeline.data.dataset.fov_num.values,number_fovs_process)
running_pipeline.data.dataset = running_pipeline.data.dataset.loc[running_pipeline.data.dataset.fov_num.isin(selected_fovs),:]


In [ ]:
specific_fov = [0]
running_pipeline.data.dataset = running_pipeline.data.dataset.loc[running_pipeline.data.dataset.fov_num.isin(specific_fov),:]

In [ ]:
running_pipeline.metadata = running_pipeline.data.collect_metadata(running_pipeline.data.dataset)
running_pipeline.grpd_fovs = running_pipeline.data.dataset.groupby('fov_num')

In [ ]:
running_pipeline.running_functions

In [ ]:
running_pipeline.analysis_parameters

In [ ]:
%%time
running_pipeline.processing_serial_fish_step()

## Run the analysis

In [ ]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/AMEXP20200914_Spinal_cordGel1')
dataset_name = 'Corrected_AMEXP20200914_Spinal_cordGel1_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'unmanaged_cluster'
start_from_preprocessed_imgs =False

In [ ]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        scheduler_port= 22217,
        dataset_path = experiment_fpath / dataset_name,
        start_from_preprocessed_imgs=start_from_preprocessed_imgs)

In [ ]:
%%time
running_pipeline.test_run_Alejandro_spinal_cord_atlas()

In [ ]:
processing_cluster_setup.kill_process()

## Re-run the microscope stitching

In [ ]:
from pysmFISH.stitching import remove_duplicated_dots_serial_graph
from pysmFISH import io

In [ ]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/AMEXP20200914_Spinal_cordGel1')
dataset_name = 'Corrected_AMEXP20200914_Spinal_cordGel1_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'unmanaged_cluster'

In [ ]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        scheduler_port= 22517,
        dataset_path = experiment_fpath / dataset_name)

## Run the required steps

In [ ]:
%%time
running_pipeline.run_setup()
running_pipeline.run_cluster_activation()
running_pipeline.run_parsing()
running_pipeline.run_required_steps()

In [ ]:
running_pipeline.client

In [ ]:
stitching_selected = 'microscope_stitched'
io.simple_output_plotting_serial(running_pipeline.experiment_fpath, stitching_selected, 
                        running_pipeline.client,
                        input_file_tag = '_decoded_fov_',
                        file_tag='cleaned_microscope_stitched')

In [ ]:
processing_cluster_setup.kill_process()

In [ ]:
from pysmFISH.data_models import Output_models
import numpy as np
import pandas as pd

In [ ]:
fill_value = np.nan
data_models = Output_models()
counts_dict = data_models.dots_counts_dict

In [ ]:
# Initialise an empty version of the counts dict
counts_dict['r_px_original'] = np.array([fill_value])
counts_dict['c_px_original'] = np.array([fill_value])
counts_dict['dot_id'] = np.array([fill_value])
counts_dict['dot_intensity'] = np.array([fill_value])
counts_dict['selected_thr'] = np.array([fill_value])

In [ ]:
fovs = running_pipeline.data.dataset.fov_num.unique()

In [ ]:
for fov in fovs:
    print(fov)
    counts_dict = data_models.dots_counts_dict
    counts_dict['r_px_original'] = np.array([fill_value])
    counts_dict['c_px_original'] = np.array([fill_value])
    counts_dict['dot_id'] = np.array([fill_value])
    counts_dict['dot_intensity'] = np.array([fill_value])
    counts_dict['selected_thr'] = np.array([fill_value])
    fov_subdataset = running_pipeline.grpd_fovs.get_group(fov).iloc[0]
    fov_subdataset_df = pd.DataFrame(fov_subdataset).T
    counts_df = pd.DataFrame(counts_dict)
    fov_subdataset_df = pd.DataFrame(fov_subdataset).T
    fov_subdataset_df = pd.concat([fov_subdataset_df]*counts_df.shape[0],axis=0).sort_index().reset_index(drop=True)
    counts_df = pd.concat([counts_df,fov_subdataset_df],axis=1)

In [ ]:
fov_subdataset_df = pd.DataFrame(fov_subdataset).T

In [ ]:
counts_df = pd.DataFrame(counts_dict)
fov_subdataset_df = pd.DataFrame(fov_subdataset).T
fov_subdataset_df = pd.concat([fov_subdataset_df]*counts_df.shape[0],axis=0).sort_index().reset_index(drop=True)
counts_df = pd.concat([counts_df,fov_subdataset_df],axis=1)

In [ ]:
counts_df

# AMEXP20200928_Spinal_cordGel2

In [ ]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/AMEXP20200928_Spinal_cordGel2')
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'reparsing_from_processing_folder'
processing_engine = 'unmanaged_cluster'

In [ ]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        scheduler_port= 22317)

In [ ]:
%%time
running_pipeline.run_setup()
running_pipeline.run_cluster_activation()

In [ ]:
running_pipeline.client

In [ ]:
%%time
running_pipeline.run_parsing()

In [16]:
processing_cluster_setup.kill_process()

distributed.worker - INFO - Process Successfully terminated


# Correct the dataset

In [ ]:
parsed_raw_data_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/AMEXP20200928_Spinal_cordGel2/AMEXP20200928_Spinal_cordGel2_img_data.zarr')
data = Dataset()
data.create_full_dataset_from_zmetadata(parsed_raw_data_fpath)

In [ ]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/AMEXP20200928_Spinal_cordGel2')
dataset_name = '220110_07_31_13_AMEXP20200928_Spinal_cordGel2_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'unmanaged_cluster'

In [ ]:
data = Dataset()
data.load_dataset(experiment_fpath / dataset_name)

In [ ]:
data.dataset.loc[:,'pipeline'] = 'smfish-serial-spinal-cord'
data.dataset.loc[:,'experiment_type'] = 'smfish-serial'
data.dataset.loc[:,'stitching_type'] = 'nuclei'
data.dataset.loc[:,'stitching_channel'] = 'DAPI'
data.dataset.loc[data.dataset.channel == 'DAPI','processing_type'] = 'nuclei'
data.dataset = data.dataset.loc[~(data.dataset.channel == 'Europium'),:]
data.dataset.loc[:,'machine'] = 'ROBOFISH1'

In [ ]:
data.save_dataset(data.dataset, experiment_fpath / 'Corrected_220110_07_31_13_AMEXP20200928_Spinal_cordGel2_img_data_dataset.parquet' )

## Run the analysis

In [11]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/AMEXP20200928_Spinal_cordGel2')
dataset_name = 'Corrected_220110_07_31_13_AMEXP20200928_Spinal_cordGel2_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'unmanaged_cluster'
start_from_preprocessed_imgs =False

In [12]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        scheduler_port= 21317,
        dataset_path = experiment_fpath / dataset_name,
        start_from_preprocessed_imgs=start_from_preprocessed_imgs)

CPU times: user 6.07 ms, sys: 3.33 ms, total: 9.41 ms
Wall time: 8.25 ms


In [ ]:
%%time
running_pipeline.test_run_Alejandro_spinal_cord_atlas()

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
distributed.worker - INFO - extra_processing_data already exist
distributed.worker - INFO - pipeline_config already exist
distributed.worker - INFO - output_figures already exist
distributed.worker - INFO - probes already exist
distributed.worker - INFO - logs already exist
distributed.worker - INFO - results already exist
distributed.worker - INFO - microscope_tiles_coords already exist
distributed.worker - INFO - notebooks already exist
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - /home/simone/mini/envs/test_d/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
distributed.deploy.ssh - INFO - Perhaps you already have a cluster running?
distributed.deploy.ssh - INFO - Hosting the HTTP server on port 22476 instead
distributed.deploy.ssh - INFO - warni

In [ ]:
processing_cluster_setup.kill_process()

## Re-run the microscope stitching

In [14]:
from pysmFISH.stitching import remove_duplicated_dots_serial_graph
from pysmFISH import io

In [15]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/AMEXP20200928_Spinal_cordGel2')
dataset_name = 'Corrected_220110_07_31_13_AMEXP20200928_Spinal_cordGel2_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'unmanaged_cluster'

In [16]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        scheduler_port= 22317,
        dataset_path = experiment_fpath / dataset_name)

CPU times: user 8.69 ms, sys: 3.93 ms, total: 12.6 ms
Wall time: 12.4 ms


## Run the required steps

In [17]:
%%time
running_pipeline.run_setup()
running_pipeline.run_cluster_activation()
running_pipeline.run_parsing()
running_pipeline.run_required_steps()

distributed.worker - INFO - extra_processing_data already exist
distributed.worker - INFO - pipeline_config already exist
distributed.worker - INFO - output_figures already exist
distributed.worker - INFO - probes already exist
distributed.worker - INFO - logs already exist
distributed.worker - INFO - results already exist
distributed.worker - INFO - microscope_tiles_coords already exist
distributed.worker - INFO - notebooks already exist
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - /home/simone/mini/envs/test_d/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
distributed.deploy.ssh - INFO - Perhaps you already have a cluster running?
distributed.deploy.ssh - INFO - Hosting the HTTP server on port 18885 instead
distributed.deploy.ssh - INFO - warnings.warn(
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -------------------------

CPU times: user 4.46 s, sys: 3.4 s, total: 7.85 s
Wall time: 2min 3s


In [18]:
running_pipeline.client

<Client: 'tcp://193.10.16.58:22317' processes=160 threads=160, memory=29.10 TiB>

In [19]:
stitching_selected = 'microscope_stitched'
io.simple_output_plotting_serial(running_pipeline.experiment_fpath, stitching_selected, 
                        running_pipeline.client,
                        input_file_tag = '_decoded_fov_',
                        file_tag='cleaned_microscope_stitched')

In [20]:
processing_cluster_setup.kill_process()

distributed.worker - INFO - Process Successfully terminated
distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


# AMEXP20201130_SpinalcordGel3

In [7]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/AMEXP20201130_SpinalcordGel3')
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 'new'
parsing_type = 'original'
processing_engine = 'unmanaged_cluster'

In [13]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        scheduler_port= 22217)

CPU times: user 379 µs, sys: 0 ns, total: 379 µs
Wall time: 386 µs


In [ ]:
%%time
running_pipeline.run_parsing_only()

distributed.worker - INFO - raw_data already exist
distributed.worker - INFO - original_robofish_logs already exist
distributed.worker - INFO - extra_processing_data already exist
distributed.worker - INFO - extra_files already exist
distributed.worker - INFO - pipeline_config already exist
distributed.worker - INFO - output_figures already exist
distributed.worker - INFO - probes already exist
distributed.worker - INFO - logs already exist
distributed.worker - INFO - results already exist
distributed.worker - INFO - microscope_tiles_coords already exist
distributed.worker - INFO - notebooks already exist
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - /home/simone/mini/envs/test_d/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
distributed.deploy.ssh - INFO - Perhaps you already have a cluster running?
distributed.deploy.ssh - INFO - Hosting the

In [15]:
processing_cluster_setup.kill_process()

distributed.worker - INFO - Process Successfully terminated


# Correct the dataset

In [20]:
# Correct the dataset file
from pysmFISH.data_models import Dataset

In [17]:
parsed_raw_data_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/AMEXP20200928_Spinal_cordGel2/AMEXP20200928_Spinal_cordGel2_img_data.zarr')
data = Dataset()
data.create_full_dataset_from_zmetadata(parsed_raw_data_fpath)

NameError: name 'Dataset' is not defined

In [21]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/AMEXP20201130_SpinalcordGel3')
dataset_name = '220111_21_25_45_AMEXP20201130_SpinalcordGel3_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'unmanaged_cluster'

In [22]:
data = Dataset()
data.load_dataset(experiment_fpath / dataset_name)

In [23]:
data.dataset.loc[:,'pipeline'] = 'smfish-serial-spinal-cord'
data.dataset.loc[:,'experiment_type'] = 'smfish-serial'
data.dataset.loc[:,'stitching_type'] = 'nuclei'
data.dataset.loc[:,'stitching_channel'] = 'DAPI'
data.dataset.loc[data.dataset.channel == 'DAPI','processing_type'] = 'nuclei'
data.dataset = data.dataset.loc[~(data.dataset.channel == 'Europium'),:]
data.dataset.loc[:,'machine'] = 'ROBOFISH1'

In [24]:
data.save_dataset(data.dataset, experiment_fpath / 'Corrected_220111_21_25_45_AMEXP20201130_SpinalcordGel3_img_data_dataset.parquet' )

## Run the analysis

In [25]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/AMEXP20201130_SpinalcordGel3')
dataset_name = 'Corrected_220111_21_25_45_AMEXP20201130_SpinalcordGel3_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'unmanaged_cluster'
start_from_preprocessed_imgs =False

In [26]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        scheduler_port= 21517,
        dataset_path = experiment_fpath / dataset_name,
        start_from_preprocessed_imgs=start_from_preprocessed_imgs)

CPU times: user 35.4 ms, sys: 16.3 ms, total: 51.7 ms
Wall time: 75.5 ms


In [27]:
%%time
running_pipeline.test_run_Alejandro_spinal_cord_atlas()

distributed.worker - INFO - extra_processing_data already exist
distributed.worker - INFO - pipeline_config already exist
distributed.worker - INFO - output_figures already exist
distributed.worker - INFO - probes already exist
distributed.worker - INFO - logs already exist
distributed.worker - INFO - results already exist
distributed.worker - INFO - microscope_tiles_coords already exist
distributed.worker - INFO - notebooks already exist
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - /home/simone/mini/envs/test_d/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
distributed.deploy.ssh - INFO - Perhaps you already have a cluster running?
distributed.deploy.ssh - INFO - Hosting the HTTP server on port 21437 instead
distributed.deploy.ssh - INFO - warnings.warn(
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -------------------------

CPU times: user 2min 19s, sys: 58.4 s, total: 3min 18s
Wall time: 6h 48min 42s


In [ ]:
processing_cluster_setup.kill_process()

## Re-run the microscope stitching

In [28]:
from pysmFISH.stitching import remove_duplicated_dots_serial_graph
from pysmFISH import io

In [15]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/AMEXP20200928_Spinal_cordGel2')
dataset_name = 'Corrected_220110_07_31_13_AMEXP20200928_Spinal_cordGel2_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'unmanaged_cluster'

In [16]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        scheduler_port= 22317,
        dataset_path = experiment_fpath / dataset_name)

CPU times: user 8.69 ms, sys: 3.93 ms, total: 12.6 ms
Wall time: 12.4 ms


## Run the required steps

In [17]:
%%time
running_pipeline.run_setup()
running_pipeline.run_cluster_activation()
running_pipeline.run_parsing()
running_pipeline.run_required_steps()

distributed.worker - INFO - extra_processing_data already exist
distributed.worker - INFO - pipeline_config already exist
distributed.worker - INFO - output_figures already exist
distributed.worker - INFO - probes already exist
distributed.worker - INFO - logs already exist
distributed.worker - INFO - results already exist
distributed.worker - INFO - microscope_tiles_coords already exist
distributed.worker - INFO - notebooks already exist
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - /home/simone/mini/envs/test_d/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
distributed.deploy.ssh - INFO - Perhaps you already have a cluster running?
distributed.deploy.ssh - INFO - Hosting the HTTP server on port 18885 instead
distributed.deploy.ssh - INFO - warnings.warn(
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -------------------------

CPU times: user 4.46 s, sys: 3.4 s, total: 7.85 s
Wall time: 2min 3s


In [18]:
running_pipeline.client

<Client: 'tcp://193.10.16.58:22317' processes=160 threads=160, memory=29.10 TiB>

In [29]:
stitching_selected = 'microscope_stitched'
io.simple_output_plotting_serial(running_pipeline.experiment_fpath, stitching_selected, 
                        running_pipeline.client,
                        input_file_tag = '_decoded_fov_',
                        file_tag='cleaned_microscope_stitched')

# DFEXP20201103_Spinal_cordGel3

In [4]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/DFEXP20201103_Spinal_cordGel3')
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 'new'
parsing_type = 'original'
processing_engine = 'unmanaged_cluster'

In [5]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        scheduler_port= 22117)

CPU times: user 187 µs, sys: 141 µs, total: 328 µs
Wall time: 343 µs


In [ ]:
%%time
running_pipeline.run_parsing_only()

distributed.worker - INFO - raw_data already exist
distributed.worker - INFO - original_robofish_logs already exist
distributed.worker - INFO - extra_processing_data already exist
distributed.worker - INFO - extra_files already exist
distributed.worker - INFO - pipeline_config already exist
distributed.worker - INFO - output_figures already exist
distributed.worker - INFO - probes already exist
distributed.worker - INFO - logs already exist
distributed.worker - INFO - results already exist
distributed.worker - INFO - microscope_tiles_coords already exist
distributed.worker - INFO - notebooks already exist
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - /home/simone/mini/envs/test_d/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
distributed.deploy.ssh - INFO - Perhaps you already have a cluster running?
distributed.deploy.ssh - INFO - Hosting the

In [30]:
processing_cluster_setup.kill_process()

distributed.worker - INFO - Process Successfully terminated


# Correct the dataset

In [10]:
# Correct the dataset file
from pysmFISH.data_models import Dataset

In [11]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/DFEXP20201103_Spinal_cordGel3')
dataset_name = '220114_05_15_15_DFEXP20201103_Spinal_cordGel3_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'unmanaged_cluster'

In [12]:
data = Dataset()
data.load_dataset(experiment_fpath / dataset_name)

In [13]:
data.dataset.loc[:,'pipeline'] = 'smfish-serial-spinal-cord'
data.dataset.loc[:,'experiment_type'] = 'smfish-serial'
data.dataset.loc[:,'stitching_type'] = 'nuclei'
data.dataset.loc[:,'stitching_channel'] = 'DAPI'
data.dataset.loc[data.dataset.channel == 'DAPI','processing_type'] = 'nuclei'
data.dataset = data.dataset.loc[~(data.dataset.channel == 'Europium'),:]
data.dataset.loc[:,'machine'] = 'ROBOFISH1'

In [14]:
data.save_dataset(data.dataset, experiment_fpath / 'Corrected_220114_05_15_15_DFEXP20201103_Spinal_cordGel3_img_data_dataset.parquet' )

## Run the analysis

In [16]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/DFEXP20201103_Spinal_cordGel3')
dataset_name = 'Corrected_220114_05_15_15_DFEXP20201103_Spinal_cordGel3_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'unmanaged_cluster'
start_from_preprocessed_imgs =False

In [17]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        scheduler_port= 21117,
        dataset_path = experiment_fpath / dataset_name,
        start_from_preprocessed_imgs=start_from_preprocessed_imgs)

CPU times: user 114 µs, sys: 34 µs, total: 148 µs
Wall time: 157 µs


In [ ]:
%%time
running_pipeline.test_run_Alejandro_spinal_cord_atlas()

distributed.worker - INFO - extra_processing_data already exist
distributed.worker - INFO - pipeline_config already exist
distributed.worker - INFO - output_figures already exist
distributed.worker - INFO - probes already exist
distributed.worker - INFO - logs already exist
distributed.worker - INFO - results already exist
distributed.worker - INFO - microscope_tiles_coords already exist
distributed.worker - INFO - notebooks already exist
distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - /home/simone/mini/envs/test_d/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
distributed.deploy.ssh - INFO - Perhaps you already have a cluster running?
distributed.deploy.ssh - INFO - Hosting the HTTP server on port 3001 instead
distributed.deploy.ssh - INFO - warnin

In [15]:
processing_cluster_setup.kill_process()

distributed.worker - INFO - Process Successfully terminated


## Re-run the microscope stitching

In [19]:
from pysmFISH.stitching import remove_duplicated_dots_serial_graph
from pysmFISH import io

In [20]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/DFEXP20201103_Spinal_cordGel3')
dataset_name = 'Corrected_220114_05_15_15_DFEXP20201103_Spinal_cordGel3_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'unmanaged_cluster'

In [21]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        scheduler_port= 22317,
        dataset_path = experiment_fpath / dataset_name)

CPU times: user 6.38 ms, sys: 5.71 ms, total: 12.1 ms
Wall time: 11.2 ms


## Run the required steps

In [22]:
%%time
running_pipeline.run_setup()
running_pipeline.run_cluster_activation()
running_pipeline.run_parsing()
running_pipeline.run_required_steps()

distributed.worker - INFO - extra_processing_data already exist
distributed.worker - INFO - pipeline_config already exist
distributed.worker - INFO - output_figures already exist
distributed.worker - INFO - probes already exist
distributed.worker - INFO - logs already exist
distributed.worker - INFO - results already exist
distributed.worker - INFO - microscope_tiles_coords already exist
distributed.worker - INFO - notebooks already exist
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - /home/simone/mini/envs/test_d/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
distributed.deploy.ssh - INFO - Perhaps you already have a cluster running?
distributed.deploy.ssh - INFO - Hosting the HTTP server on port 18566 instead
distributed.deploy.ssh - INFO - warnings.warn(
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -------------------------

CPU times: user 3.43 s, sys: 2.68 s, total: 6.12 s
Wall time: 1min 52s


In [23]:
running_pipeline.client

<Client: 'tcp://193.10.16.58:22317' processes=160 threads=160, memory=29.10 TiB>

In [24]:
stitching_selected = 'microscope_stitched'
io.simple_output_plotting_serial(running_pipeline.experiment_fpath, stitching_selected, 
                        running_pipeline.client,
                        input_file_tag = '_decoded_fov_',
                        file_tag='cleaned_microscope_stitched')

## Run the analysis

In [11]:
# Enter the required parameters
experiment_fpath = Path('/rawa/sl/fish_rawdata/SpinalCord/AMEXP20200928_Spinal_cordGel2')
dataset_name = 'Corrected_220110_07_31_13_AMEXP20200928_Spinal_cordGel2_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'unmanaged_cluster'
start_from_preprocessed_imgs =False

In [12]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        scheduler_port= 21317,
        dataset_path = experiment_fpath / dataset_name,
        start_from_preprocessed_imgs=start_from_preprocessed_imgs)

CPU times: user 6.07 ms, sys: 3.33 ms, total: 9.41 ms
Wall time: 8.25 ms


In [ ]:
%%time
running_pipeline.test_run_Alejandro_spinal_cord_atlas()

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
distributed.worker - INFO - extra_processing_data already exist
distributed.worker - INFO - pipeline_config already exist
distributed.worker - INFO - output_figures already exist
distributed.worker - INFO - probes already exist
distributed.worker - INFO - logs already exist
distributed.worker - INFO - results already exist
distributed.worker - INFO - microscope_tiles_coords already exist
distributed.worker - INFO - notebooks already exist
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - /home/simone/mini/envs/test_d/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
distributed.deploy.ssh - INFO - Perhaps you already have a cluster running?
distributed.deploy.ssh - INFO - Hosting the HTTP server on port 22476 instead
distributed.deploy.ssh - INFO - warni

In [25]:
processing_cluster_setup.kill_process()

distributed.worker - INFO - Process Successfully terminated
distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


## Select the fovs to process to test the pipeline

__NB__: error in the yaml file adjust    
running_pipeline.metadata['experiment_type'] = 'smfish-serial'  
running_pipeline.metadata['pipeline'] = 'smfish-serial'  

In [ ]:
running_pipeline.data.dataset.processing_type.unique()

In [ ]:
running_pipeline.running_functions

In [ ]:
- new setup with robofish1 best
- old room
    - bottom-right
    - bootm-left

In [ ]:
running_pipeline.metadata['machine'] = 'ROBOFISH1'
running_pipeline.data.dataset.loc[:,'machine'] = 'ROBOFISH1'

In [ ]:
running_pipeline.tiles_org.reference_corner_fov_position

In [ ]:
running_pipeline.determine_tiles_organization_before_room_reorganisation() 

In [ ]:
running_pipeline.tiles_org.reference_corner_fov_position = 'top-left'

In [ ]:
from pysmFISH.stitching import stitch_using_coords_general

In [ ]:
all_futures = []
for fpath in (running_pipeline.experiment_fpath / 'results').glob('*_microscope_stitched_cleaned_df*'):
    future = running_pipeline.client.submit(stitch_using_coords_general,
                                decoded_df=fpath,
                                tile_corners_coords_pxl = running_pipeline.tiles_org.tile_corners_coords_pxl,
                                reference_corner_fov_position=running_pipeline.tiles_org.reference_corner_fov_position,
                                metadata=running_pipeline.metadata,
                                tag = 'microscope_stitched')

    all_futures.append(future)

_ = running_pipeline.client.gather(all_futures)  


In [ ]:
%%time
remove_duplicated_dots_serial_graph(running_pipeline.experiment_fpath,running_pipeline.data.dataset,running_pipeline.tiles_org,
                                    running_pipeline.same_dot_radius_duplicate_dots, 
                                    running_pipeline.stitching_selected, running_pipeline.client)
 

In [ ]:
t_org = organize_square_tiles_old_room_test(running_pipeline.experiment_fpath,
                                           running_pipeline.data.dataset,
                                           running_pipeline.metadata,
                                           round_num=1)

In [ ]:
t_org.extract_microscope_coords()

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
t_org.x_coords[0:10]

In [ ]:
t_org.y_coords[0:10]

In [ ]:
import numpy as np

In [ ]:
d = np.array([1])

In [ ]:
all_fovs = list(running_pipeline.grpd_fovs.groups.keys())

In [ ]:
import numpy as np
from pysmFISH import io
from pysmFISH.dots_calling import osmFISH_peak_based_detection,osmFISH_dots_thr_selection,osmFISH_dots_mapping

In [ ]:
preprocessed_zarr_fpath = running_pipeline.experiment_fpath / 'AMEXP20200914_Spinal_cordGel1_preprocessed_img_data.zarr'

In [ ]:
running_pipeline.data.load_dataset(dataset_fpath=running_pipeline.experiment_fpath / dataset_name)
running_pipeline.grpd_fovs = running_pipeline.data.dataset.groupby('fov_num')

In [ ]:
for v in np.arange(0,75):
    print(v)
    fov_subdataset = running_pipeline.grpd_fovs.get_group(0).iloc[v]
    filt_out = io.load_general_zarr(fov_subdataset,preprocessed_zarr_fpath,tag='preprocessed_data')
    img = filt_out[0]
    counts= osmFISH_peak_based_detection(img,fov_subdataset,running_pipeline.analysis_parameters['fish'])

    

In [ ]:
fov_subdataset = running_pipeline.grpd_fovs.get_group(0).iloc[14]
filt_out = io.load_general_zarr(fov_subdataset,preprocessed_zarr_fpath,tag='preprocessed_data')
img = filt_out[0]

In [ ]:
counting_parameters_dict = {
                            'min_distance': running_pipeline.analysis_parameters['fish']['CountingFishMinObjDistance'],
                            'min_obj_size': running_pipeline.analysis_parameters['fish']['CountingFishMinObjSize'],
                            'max_obj_size':  running_pipeline.analysis_parameters['fish']['CountingFishMaxObjSize'],
                            'num_peaks_per_label':  running_pipeline.analysis_parameters['fish']['CountingFishNumPeaksPerLabel']
                                }

counts = osmFISH_dots_thr_selection(img,counting_parameters_dict)
counts.counting_graph()
counts.thr_identification()

In [ ]:
dots = osmFISH_dots_mapping(img,counts.selected_thr,counting_parameters_dict)

In [ ]:
dots.intensity_array

In [ ]:
create_general_analysis_config_file('/fish/config_db/')